In [1]:
%matplotlib inline
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from components.AttractorLayer import AttractorLayer
from components.RadarLoader import RadarLoader

seq_name = "2020-02-28-13-13-43"
%run carrada_utils/scripts/set_path.py 'components/carrada_datasets/'

In [2]:
data_loader = RadarLoader(seq_name)
data, size_bf = data_loader.get_range_angle_stream_data(clip_and_normalize=True, resize=(64,64))
data_diff_normalized = data_loader.get_range_angle_stream_data_differentiated(clip_and_normalize=True, resize=(64,64))
raw_camera_data = data_loader.get_color_image_datastream(resize=(64,64))

In [3]:

TAU = 1  # TODO: I don't know whether this is important
INTENSITY =1.0
BETA =300  # TODO : Check this
SIGMA =15.0 # TODO: Check this
SHIFT = 0.0 # TODO : Check this
CUTOFF_DIST = 64 # TODO : Change this
X_EYE = 1.0
Y_EYE = 1.0
K_INHIB = 5.0


In [4]:


def get_annotations(differentiated=False):
    annotations = data_loader.get_annotations()
    print("HERE")
    dense_ = []
    for entry in annotations:
        arrr = np.zeros(size_bf)
        for ittem in  annotations[entry]:
            points =annotations[entry][ittem]['range_angle']["dense"]
            annot_pts = np.array(points)
            arrr[annot_pts[:,0], annot_pts[:,1]] = 0.5
        resized_annot = cv2.resize(arrr, (64, 64))
        dense_.append(resized_annot.copy())
    sparse_ = []
    sparse_mean_points=[]
    sparse_mp_visualization=[]
    for entry in annotations:
        arrr = np.zeros(size_bf)
        arrr_mp = np.zeros(size_bf)
        s__ = []
        for ittem in  annotations[entry]:
            points =annotations[entry][ittem]['range_angle']["sparse"]
            annot_pts = np.array(points)
            print(np.array(points))
            mean_point=np.sum(np.array(points), axis=0) // len(np.array(points))
            s__.append(mean_point)
            # PRINT HERE? TO SEE THE POINTS? THEN CALCULATE THE MEAN POINT
            arrr[annot_pts[:,0], annot_pts[:,1]] = 0.5
            arrr_mp[mean_point[0], mean_point[1]] = 1.0
        sparse_mean_points.append(s__.copy())
        resized_annot = cv2.resize(arrr, (64, 64))
        sparse_.append(resized_annot.copy())
        sparse_mp_visualization.append(cv2.resize(arrr_mp, (64,64)).copy())
        print("next")
    box_ = []
    for entry in annotations:
        arrr = np.zeros(size_bf)
        for ittem in  annotations[entry]:
            points =annotations[entry][ittem]['range_angle']["box"]
            annot_pts = np.array(points)
            arrr[annot_pts[:,0], annot_pts[:,1]] = 0.5
        resized_annot = cv2.resize(arrr, (64, 64))
        box_.append(resized_annot.copy())
    if differentiated:
        return dense_[1:], sparse_[1:], box_[1:]
    else:
        return dense_, sparse_, box_, sparse_mean_points, sparse_mp_visualization
# TODO : Plot annotations

In [5]:
s = [[23,4],[123,5],[232,54]]
s_ = np.array(s)
np.sum(s_, axis=0)
len(s_)

3

In [6]:
attr_layer = AttractorLayer(
        tau=TAU,
        intensity=INTENSITY,
        cutoff_dist=CUTOFF_DIST,
        sigma=SIGMA,
        shift=SHIFT,
        beta=BETA,
        k=K_INHIB,
        clip=True,
        x_eye=X_EYE,
        y_eye=Y_EYE,
)

attr_layer.set_weights()
attr_layer.save_network(file_path="components/network_weights/threes_win.npy")

#attr_layer.load_network(file_path="components/network_weights/9_net.npy")

In [7]:
neuron_act = []
for data_entry in data_diff_normalized:
    neuron_act.append(
        attr_layer.forward_pass_visualization(
            data_entry.flatten(), number_of_passes=1
        )[0]
    )

In [8]:
dense_, sparse_, box_, sp_mp_, sp_mp_vis_ = get_annotations(differentiated=True)


HERE
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
next
[[ 80 128]
 [ 79 133]
 [ 79 129]
 [ 78 131]]
next
[[ 81 129]]
next
[[ 83 129]
 [ 82 130]
 [ 81 129]
 [ 82 129]
 [ 81 131]]
next
[[ 83 131]
 [ 82 131]]
next
[[ 86 131]
 [ 86 130]
 [ 85 129]]
next
[[ 87 130]
 [ 88 129]
 [ 87 129]
 [ 86 130]]
next
[[ 88 129]
 [ 87 129]]
next
[[ 91 128]
 [ 91 128]
 [ 91 128]
 [ 89 128]]
next
[[ 93 129]
 [ 92 129]
 [ 90 129]
 [ 91 130]
 [ 90 131]]
next
[[ 93 129]
 [ 94 130]
 [ 91 128]]
next
[[ 94 129]
 [ 94 129]
 [ 93 128]
 [ 92 129]]
next
[[ 97 133]
 [ 95 132]]
next
[[ 98 130]
 [ 97 129]
 [ 95 128]]
next
[[ 99 130]
 [ 99 130]
 [ 98 130]
 [ 98 129]
 [ 98 130]
 [ 97 130]
 [ 97 130]
 [ 97 130]]
next
[[101 131]
 [100 131]
 [10

ValueError: not enough values to unpack (expected 5, got 3)

In [ ]:
CUTOFF = 0.5
mIoU_dense = []
mDice_dense = []
mIoU_sparse = []
mDice_sparse = []
for i in range(0,len(neuron_act)-1):
    A_acts = neuron_act[i].copy()
    A_acts[A_acts < 0.5] = 0
    A_acts[A_acts > 0.4] = 1
    B_gt_dense = dense_[i].copy() * 2
    B_gt_sparse = sparse_[i].copy() * 2
     
    A_B_intersection_dense = A_acts.copy()
    A_B_intersection_dense[B_gt_dense == 1.0] = 1.0
    A_B_intersection_dense[B_gt_dense != 1.0] = 0.0
    
    A_B_union_dense = B_gt_dense.copy()
    A_B_union_dense[A_acts > 0] = 1.0
    
    mIoU_dense.append(np.sum(A_B_intersection_dense) / np.sum(A_B_union_dense))
    mDice_dense.append(np.sum(A_B_intersection_dense) * 2 / (np.sum(B_gt_dense) + np.sum(A_acts)))
    
     
    A_B_intersection_sparse = A_acts.copy()
    A_B_intersection_sparse[B_gt_sparse == 1.0] = 1.0
    A_B_intersection_sparse[B_gt_sparse != 1.0] = 0.0
    
    A_B_union_sparse = B_gt_sparse.copy()
    A_B_union_sparse[A_acts > 0] = 1.0
    
    mIoU_sparse.append(np.sum(A_B_intersection_sparse) / np.sum(A_B_union_sparse))
    mDice_sparse.append(np.sum(A_B_intersection_sparse) * 2 / (np.sum(B_gt_sparse) + np.sum(A_acts)))
    
    
title_string = f"Losses \n        mIoU_dense: {np.mean(mIoU_dense[40:]):.6f}\n      mDice_dense: {np.mean(mDice_dense[40:]):.6f}\n       mIoU_sparse: {np.mean(mIoU_sparse[40:]):.6f}\n      mDice_sparse: {np.mean(mDice_sparse[40:]):.6f}\n"
print(title_string)

In [ ]:
fig, (ax1, ax2, ax3, ax4, ax5)= plt.subplots(1, 5, figsize=(18,5), dpi=100)
fig.suptitle(title_string, fontsize=10)
rgb_img= np.zeros((64,64,3))
rgb_img[:,:,0] = neuron_act[100]
rgb_img[:,:,1] = dense_[100]

ax1.tick_params(axis='both',labelsize=8)
ax1.set_title("Neuron Activities with Annotation",fontsize=10)
ax2.tick_params(axis='both',labelsize=8)
ax2.set_title("Neuron Activities",fontsize=10)
ax3.tick_params(axis='both',labelsize=8)
ax3.set_title("Differentiated Input",fontsize=10)
ax4.tick_params(axis='both',labelsize=8)
ax4.set_title("Raw Data Input",fontsize=10)
ax5.tick_params(axis='both',labelsize=8)
ax5.set_title("Raw Camera RGB Data",fontsize=10)

cell_act_map_w_annot = ax1.imshow(rgb_img, cmap='hot', interpolation='none')
cell_act_map = ax2.imshow(np.array(neuron_act[100]), cmap='hot', interpolation='none')
input_diff_map = ax3.imshow(np.array(data_diff_normalized[100]), cmap='hot', interpolation='none')
input_map = ax4.imshow(np.array(data[100]), cmap='hot', interpolation='none')
rgb_cam_data = ax5.imshow(np.array(raw_camera_data[100]), interpolation='none')

plt.show()


In [ ]:


fig, (ax1, ax2, ax3, ax4, ax5)= plt.subplots(1, 5, figsize=(18,5), dpi=400)
fig.suptitle(title_string, fontsize=10)

annotated_neuron_act = np.zeros((len(neuron_act), 64,64,3))
annotated_neuron_act[:,:,:,0] = neuron_act
annotated_neuron_act[:,:,:,1] = dense_

ax1.tick_params(axis='both',labelsize=8)
ax1.set_title("Neuron Activities with Annotation",fontsize=10)
ax2.tick_params(axis='both',labelsize=8)
ax2.set_title("Neuron Activities",fontsize=10)
ax3.tick_params(axis='both',labelsize=8)
ax3.set_title("Differentiated Input",fontsize=10)
ax4.tick_params(axis='both',labelsize=8)
ax4.set_title("Raw Data Input",fontsize=10)
ax5.tick_params(axis='both',labelsize=8)
ax5.set_title("Raw Camera RGB Data",fontsize=10)

cell_act_map_w_annot = ax1.imshow(annotated_neuron_act[0], cmap='hot', interpolation='none')
cell_act_map = ax2.imshow(np.array(neuron_act[0]), cmap='hot', interpolation='none')
input_diff_map = ax3.imshow(np.array(data_diff_normalized[0]), cmap='hot',interpolation='none')
input_map = ax4.imshow(np.array(data[1]), cmap='hot', interpolation='none')
rgb_cam_data = ax5.imshow(np.array(raw_camera_data[1]), interpolation='none')


def init():
    
    cell_act_map_w_annot.set_data(annotated_neuron_act[0])
    cell_act_map.set_data(np.array(neuron_act[0]))
    input_diff_map.set_data(np.array(data_diff_normalized[0]))
    input_map.set_data(np.array(data[1]))
    rgb_cam_data.set_data(np.array(raw_camera_data[1]))
    return [cell_act_map_w_annot, cell_act_map, input_diff_map, input_map, rgb_cam_data]

# animation function.  This is called sequentially
def animate(i):
    cell_act_map_w_annot.set_data(annotated_neuron_act[i])
#     arr = neuron_act[i]/np.max(neuron_act[i])
#     arr[arr < 0.4] = 0
    cell_act_map.set_data(np.array((neuron_act[i])))
    input_diff_map.set_data(np.array(data_diff_normalized[i]))
    input_map.set_data(np.array(data[i+1]))
    rgb_cam_data.set_data(np.array(raw_camera_data[i+1]))
    return [cell_act_map_w_annot, cell_act_map, input_diff_map, input_map, rgb_cam_data]


anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=1, blit=True)

anim.save('animations/final_anim1_sp_cam.gif', fps=10)


#fig.colorbar(cell_dists_map)

#plt.show()